In [1]:
import os
from dotenv import load_dotenv

import pandas as pd
from langchain import hub
from langchain_openai import OpenAIEmbeddings
from langchain_core.documents import Document
from langchain_chroma import Chroma
from langchain_openai import ChatOpenAI
from langchain_core.runnables import RunnablePassthrough
from langchain_core.prompts import ChatPromptTemplate
from langchain_core.output_parsers import StrOutputParser

c:\Users\cyj1226\anaconda3\envs\rag\Lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
# API 불러오기
load_dotenv()
OPENAI_API_KEY = os.getenv("OPENAI_API_KEY")

In [4]:
# 학생 이름을 기준 그룹화 함수
def create_grouped_documents(csv_path: str) -> list[Document]:
    """
    CSV 파일을 로드하여 학생 이름별로 출결 기록을 그룹화하고
    LangChain Document 객체 리스트로 반환합니다.

    Args:
        csv_path: 출결 CSV 파일의 경로.

    Returns:
        Document 객체 리스트. 각 Document는 한 학생의 전체 기록을 담습니다.
    """
    # 1. CSV 파일 로드
    df = pd.read_csv(csv_path, encoding='cp949')

    # 필요한 컬럼만 선택하고 NaN 값은 빈 문자열로 대체 (문자열 결합 시 오류 방지)
    required_cols = ['이름', '사유', '날짜', '부재시간', '상태']
    if not all(col in df.columns for col in required_cols):
        print(f"오류: CSV 파일에 필요한 컬럼 ({required_cols}) 중 일부가 누락되었습니다.")
        return []

    df = df[required_cols].fillna('')

    # Document 객체를 저장할 리스트
    documents = []

    # 2. '이름' 컬럼을 기준으로 그룹화
    grouped = df.groupby('이름')

    # 3. 각 학생 그룹을 하나의 긴 텍스트 Document로 변환
    for name, group_df in grouped:
        # 학생별 기록을 문자열로 변환 (날짜, 부재시간, 상태만 포함)
        # '이름' 컬럼은 메타데이터로 사용하기 때문에 텍스트 내용에서는 제외합니다.
        record_strings = []
        for index, row in group_df.iterrows():
            record = (
                f"사유: {row['사유']}, "
                f"날짜: {row['날짜']}, "
                f"상태: {row['상태']}, "
                f"부재시간: {row['부재시간']}"
            )
            record_strings.append(record)

        # 모든 기록을 줄 바꿈으로 연결하여 하나의 긴 텍스트 생성
        full_records_text = "\n".join(record_strings)

        # 최종 Document 객체 생성
        document = Document(
            page_content=(
                f"학생 이름: {name}\n\n"
                f"--- 전체 출결 기록 시작 ---\n"
                f"{full_records_text}"
            ),
            # 메타데이터에 핵심 정보 저장 (검색 시 활용 가능)
            metadata={'학생이름': name, '총기록수': len(group_df)}
        )
        documents.append(document)

    return documents

In [6]:
# 학생별 Document 리스트 생성
attendance_documents = create_grouped_documents(csv_path='../data/데싸 5기 일정표.csv')

# Chroma DB에 전달
if attendance_documents:
    EMBEDDING_MODEL = OpenAIEmbeddings(model='text-embedding-3-small')
    DB_PATH = './grouped_attendance_db' # DB 생성 경로

    # 생성된 Document 리스트를 DB에 전달 (Text Splitter 없이 직접 임베딩)
    vectorstore = Chroma.from_documents(
        documents=attendance_documents,
        embedding=EMBEDDING_MODEL,
        persist_directory=DB_PATH
    )

# retriever 지정
retriever = vectorstore.as_retriever(search_type='similarity', search_kwargs={'k': 10})

In [7]:
# llm 모델 로드
llm = ChatOpenAI(model='gpt-4o-mini')

# langchain hub에서 rag 프롬프트 호출
prompt = hub.pull('rlm/rag-prompt')

In [8]:
# chain 구축
rag_chain = (
    {'context': retriever, 'question': RunnablePassthrough()}
    | prompt
    | llm
    | StrOutputParser()
)

In [9]:
answer = rag_chain.invoke('손호진님의 모든 조퇴/휴가 일정을 알려줘')
print(answer)

손호진님의 조퇴 일정은 다음과 같습니다: 2025-07-28, 2025-09-05, 2025-09-15, 2025-10-15, 2025-10-17, 2025-10-22. 휴가는 2025-08-22와 2025-09-08에 각각 신청되었습니다.
